I will use the data from Kaggle's [ALAKSA2 image steganalysis competition](https://www.kaggle.com/c/alaska2-image-steganalysis/overview)

# Set up

In [1]:
from __future__ import absolute_import, division

In [2]:
!rm sample_data/ -r
from google.colab import drive
drive.mount('/content/drive')

rm: cannot remove 'sample_data/': No such file or directory
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
myDrive = "/content/drive/'My Drive'"
!cp {myDrive}"/minisample" ./minisample -r

# input_directory = 'C:\\Users\\Echo9\\PycharmProjects\\Stego'
input_directory = '/content'
subdirectory='/minisample'

%cd {input_directory}

/content


## Dependencies

In [4]:
# Some utilities
import os, glob, itertools, datetime
from typing import Dict, Tuple, Optional, List, Generator

# Maths
import numpy as np
import pandas as pd; print('Pandas ', pd.__version__, pd.__git_version__)

# Viz
import matplotlib.pyplot as plt

# visuals
from IPython.display import display
from PIL import Image

Pandas  1.0.5 b687cd4d9e520666a956a60849568a98dd00c672


In [5]:
import tensorflow as tf;
AUTOTUNE = tf.data.experimental.AUTOTUNE

display("TensorFlow:  {}".format(tf.__version__),
        'TF git version:  {}'.format(tf.__git_version__))

# Load the TensorBoard notebook extension.
%load_ext tensorboard

'TensorFlow:  2.2.0'

'TF git version:  v2.2.0-0-g2b96f3662b'

In [6]:
!pip install {myDrive}/packages
import stego

Processing ./drive/My Drive/packages
  Created wheel for stego-pack: filename=stego_pack-0.0.2-cp36-none-any.whl size=6960 sha256=a511740d8f75bc870a6ce63b2f87dab9ace2a729733327af6ed2e7875047f233
  Stored in directory: /tmp/pip-ephem-wheel-cache-fe8ch04y/wheels/66/2a/c0/f227d155814a8369d20ccfd7ed8624f0ad88273d430a431c6e
Successfully built stego-pack
  Found existing installation: stego-pack 0.0.2
    Uninstalling stego-pack-0.0.2:
      Successfully uninstalled stego-pack-0.0.2


## Check for GPU / TPU

In [7]:
rig =stego.Rig()

INFO:tensorflow:Initializing the TPU system: grpc://10.22.20.10:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.22.20.10:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [8]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, 
and then re-execute this cell.


# Data retrival and preprocesing

In [9]:
%ls  .{subdirectory}

Cover/  JMiPOD/  JUNIWARD/  minisample/  UERD/


### Download the images

In [10]:
train_images_list = ['Cover','JMiPOD', 'JUNIWARD', 'UERD']
test_images_dict = ['Test']
data_downloader = stego.GetData(class_names=train_images_list,
                                    input_directory=input_directory,
                                    subdirectory=subdirectory)
train_img_generator = data_downloader.\
                        img_batch(10, (512, 512), 'training',
                                  preprocessing_function=tf.image.rgb_to_yuv)
valid_img_generator = data_downloader.\
                        img_batch(10, (512, 512), 'validation',
                                  preprocessing_function=tf.image.rgb_to_yuv)

training:
Found 1171 images belonging to 4 classes.
validation:
Found 501 images belonging to 4 classes.


In [11]:
total_train = 1171; total_val = 501

## Image sample after preprocesing 

our preprocessing had this steps:
1. Read data.
1. Decode, convert to grid format as per their RGB content.
1. Convert into floating point tensors.
1. Rescale [0, 255] → [0, 1]
1. Transform RGB to YUV color space.

#### Sample of images

</p>Our eyes catch better the brightness than the color. JPEG will consider this and preserve more information of the luminescence by having smaller coefficients for in the quantization table of the luminance. The JPEG algorithm quantizes the image (divides by the quantization table) the luma layer has higher values —most of the time.
</p>

The redness in these sample images it's because it's interpreting the luma layer as red, so we can deduce that the transformation worked.

## Display each space separated

# Training

In [12]:
tf.keras.backend.clear_session()

In [13]:
tf.config.optimizer.set_experimental_options({"auto_mixed_precision": True})
print('Mixed precision enabled')

Mixed precision enabled


In [14]:
rig = stego.HardwareAssistant.Rig()
print(rig)

INFO:tensorflow:Initializing the TPU system: grpc://10.22.20.10:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.22.20.10:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Running on TPU grpc://10.22.20.10:8470 
replicas: 8


TypeError: ignored

## Model: Baseline

### Variables

In [15]:
# ———————— Dependencies
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (Input, Dense, Conv2D, Flatten, Dropout,
                                    MaxPooling2D, AveragePooling2D,
                                     BatchNormalization)
import datetime
# ———————— Variables
KERNEL_1 = STRIDE_1 = 8
KERNEL_2 = STRIDE_2 = 4
IMG_HEIGHT = IMG_WIDTH = 512
BATCH_SIZE = 32
EPOCHS = 15
CLASS_MODE='categorical'
VERBOSE = 1
steps_per_epoch = total_train//BATCH_SIZE
validation_steps = total_val // BATCH_SIZE
# ———————— Parameters
compiler_params = {
    'loss':'categorical_crossentropy',
    'optimizer':'nadam',
    'metrics':['AUC', 'categorical_crossentropy']
}

# ———————— Callbacks
## Tensorboard
tensorboard_callback = tf.keras.callbacks.TensorBoard(**{
    'log_dir': input_directory+'/logs/' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S'),
    'histogram_freq': 1
})
## Checkpoints
checkpoint_filepath = input_directory+'/logs/checkpoint/'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_acc',
    mode='max',
    save_best_only=True)

# ———————— Compile and run
def runModel(model, compiler_params=compiler_params,
             train_Generator = train_img_generator,
             valid_Generator = valid_img_generator,
             epochs = EPOCHS,
             steps_per_epoch = steps_per_epoch,
             validation_steps= validation_steps):

    model.compile(**compiler_params)

    model.fit(
        train_img_generator,
        epochs=EPOCHS,
        steps_per_epoch=steps_per_epoch,
        validation_data=valid_img_generator,
        validation_steps=validation_steps,
        callbacks=[tensorboard_callback, model_checkpoint_callback]
    )

### Arquitecture

In [16]:
# ———————— Architecture
def baseLine_arquitect(cNames=4):
    # tf.reset_default_graph()
    tf.keras.backend.clear_session()
    
    model = Sequential()
    n_classes = cNames
    
    # Convolutional layers
    ## First layer
    model.add(Conv2D(2**10, (KERNEL_1), strides=(STRIDE_1), padding='same',
                     activation='elu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3),
                     activity_regularizer='l1'))
    model.add(BatchNormalization())
    ## Second layer
    model.add(Conv2D(2**11, (KERNEL_1), strides=(STRIDE_1), padding='same',
                     activation='elu'))
    model.add(BatchNormalization()); model.add(Dropout(0.20))    
    ## Third layer
    model.add(Conv2D(2**8, (KERNEL_1), strides=(STRIDE_1), padding='same',
                     activation='elu'))
    model.add(BatchNormalization()); model.add(Dropout(0.30))
    
    # Flat land
    model.add(Flatten())
    model.add(Dense(2**5, activation='elu'))
    model.add(Dropout(0.50))
    model.add(Dense(2**4, activation='elu'))
    model.add(Dropout(0.50))
    model.add(Dense(n_classes, activation='softmax'))

    return model

baseLine = baseLine_arquitect()
baseLine.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 64, 64, 1024)      197632    
_________________________________________________________________
batch_normalization (BatchNo (None, 64, 64, 1024)      4096      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 8, 8, 2048)        134219776 
_________________________________________________________________
batch_normalization_1 (Batch (None, 8, 8, 2048)        8192      
_________________________________________________________________
dropout (Dropout)            (None, 8, 8, 2048)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 1, 1, 256)         33554688  
_________________________________________________________________
batch_normalization_2 (Batch (None, 1, 1, 256)         1

In [17]:
!ls '/content/logs/'

In [19]:
baseLine_history = runModel(baseLine)

UnimplementedError: ignored

## 

In [ ]:
%tensorboard --logdir logs

## Model: GAN inspired

### Variables

In [ ]:
augmented_train_batches = (
    train_dataset
    # Only train on a subset, so you can quickly see the effect.
    .take(6000)
    .cache()
    .map(augment, num_parallel_calls=AUTOTUNE)
    .batch(BATCH_SIZE)
    .prefetch(AUTOTUNE)
)

aug_history = model_with_aug.fit(augmented_train_batches,
                                 epochs=50,
                                 validation_data=validation_batches)

In [ ]:
# ———————— Dependencies
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (Input, Dense, Conv2D, Flatten, Dropout,
                                    MaxPooling2D, AveragePooling2D,
                                     BatchNormalization)
import datetime
# ———————— Variables
KERNEL_1 = STRIDE_1 = 8
KERNEL_2 = STRIDE_2 = 4
IMG_HEIGHT = IMG_WIDTH = 512
BATCH_SIZE = 64
EPOCHS = 15
CLASS_MODE='categorical'
VERBOSE = 1
steps_per_epoch = total_train//BATCH_SIZE
validation_steps = total_val // BATCH_SIZE
# ———————— Parameters

compiler_params = {
    'loss':'categorical_crossentropy',
    'optimizer':'nadam',
    'metrics':['AUC']
}

# ———————— Callbacks
## Tensorboard
tensorboard_callback = tf.keras.callbacks.TensorBoard(**{
    'log_dir': input_directory+'/logs/' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S'),
    'histogram_freq': 1
})
## Checkpoints
checkpoint_filepath = input_directory+'/logs/checkpoint/'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_acc',
    mode='max',
    save_best_only=True)

# ———————— Compile and run
def runModel(model, compiler_params=compiler_params,
             train_Generator = train_img_generator,
             valid_Generator = valid_img_generator,
             epochs = EPOCHS,
             steps_per_epoch = steps_per_epoch,
             validation_steps= validation_steps):

    model.compile(**compiler_params)

    model.fit_generator(
        train_Generator,
        epochs=EPOCHS,
        steps_per_epoch=steps_per_epoch,
        validation_data=valid_Generator,
        validation_steps=validation_steps,
        callbacks=[tensorboard_callback,
                   model_checkpoint_callback]
    )

In [ ]:
# ———————— Variables
KERNEL_SIZE = STRIDE_LEN = 8
IMG_HEIGHT = IMG_WIDTH = 512
BATCH_SIZE = 16
EPOCS = 10
CLASS_MODE='categorical'
VERBOSE = 1

# ———————— Architecture
def discriminatorModel(depth=64, p=0.4):
    from tensorflow.keras.models import Model
    # Define inputs
    image = Input((STRIDE_LEN,IMG_HEIGHT,3))
    
    # Convolutional layers
    conv1 = Conv2D(depth*1, 5, strides=2, 
                   padding='same', activation='relu')(image)
    conv1 = Dropout(p)(conv1)
    
    conv2 = Conv2D(depth*2, 5, strides=2, 
                   padding='same', activation='relu')(conv1)
    conv2 = Dropout(p)(conv2)
    
    conv3 = Conv2D(depth*4, 5, strides=2, 
                   padding='same', activation='relu')(conv2)
    conv3 = Dropout(p)(conv3)
    
    conv4 = Conv2D(depth*8, 5, strides=1, 
                   padding='same', activation='relu')(conv3)
    conv4 = Flatten()(Dropout(p)(conv4))
    
    # Output layer
    prediction = Dense(1, activation='sigmoid')(conv4)
    
    # Model definition
    model = Model(inputs=image, outputs=prediction)
    
    return model


discriminator_model = discriminatorModel()
discriminator_model.summary()

In [ ]:
runModel(discriminator_model)

## Clear process list

In [ ]:
tf.reset_default_graph()
# tf.keras.backend.clear_session()

In [ ]:
!ps -aux|grep ipykernel_launcher  # proces that use GPU

# Visualizing results

In [ ]:
!pip install -q git+https://github.com/tensorflow/docs

import tensorflow_docs as tfdocs
import tensorflow_docs.modeling
import tensorflow_docs.plots

In [ ]:
plotter = tfdocs.plots.HistoryPlotter()
plotter.plot({"Augmented": baseLine_history, "Non-Augmented": no_aug_history}, metric = "accuracy")
plt.title("Accuracy")
plt.ylim([0.75,1])

In [ ]:
acc = bl_res.history['accuracy']
val_acc = bl_res.history['val_accuracy']

loss=bl_res.history['loss']
val_loss=bl_res.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

## Tensorboard